In [54]:
%reset -f

import numpy as np
import pandas
import sympy
from termcolor import colored

#### inputs
stream_direction = ["in","out","out"]
component_labels = ["Flow rate","Oxygen","Nitrogen","Argon"]
fraction_prefix = "y"

y1N = 0.7808
F2 = 100
y2O = 0.9
y1O = 0.2098
y2N = 0.02
y3A = 0


####

Ns = len(stream_direction)
Nc = len(component_labels)-1

#### solving procedure

s_labels = []
for i in range(Ns):
    s_labels.append("stream-" + str(i+1) + "_" + stream_direction[i])

V_empty=np.empty([Ns,Nc+1],dtype=object)
df = pandas.DataFrame(V_empty, index=s_labels, columns=component_labels)

# Prepare an empty list for storing all variable names
all_variables = []

# Loop the streams to generate flow rate names

for i in range(Ns):
    all_variables.append(component_labels[0][0] + str(i+1))

for i in range(len(component_labels[1:])):  #starting from 1 because flowrate is not included
    for j in range(Ns):
        all_variables.append(fraction_prefix + str(j+1) + component_labels[i+1][0])
        
# Declare the symbols based on the variables
all_symbols = sympy.symbols(all_variables)

# Create a duplicate array for inserting the variables
V_initial = V_empty.copy()

given_variables = []


# Loop all the variables and insert into the new flow table (V_initial)
for i in range(len(all_variables)):
   
    # Find the row index for inserting variables, based on the stream number
    
    index_row = int(all_variables[i][1]) - 1 
    
     # if the 1st string of the variable name equals to the "F" (or any name you defined)
    # the column index should be zero (1st column)
    
    if all_variables[i][0] == component_labels[0][0]:
        index_column = 0
        
        # if not flowrate, it should be components
        
    else:
            for j in range(len(component_labels[1:])):
             
            #find the column index by matching the variable names and component labels
                if all_variables[i][2] == component_labels[j+1][0]:
                    index_column = j+1
   
    # Check if the variable has been given values (input)
    if all_variables[i] in globals():
      
    # insert the value into the new array
        V_initial[index_row,index_column] = eval(all_variables[i])
        given_variables.append(all_variables[i])
    # If the variable has not been given (wait to be resolved)
    else:
     
    # insert a symbol into the new array
        V_initial[index_row,index_column] = all_symbols[i]

    # Show the flow table       
df = pandas.DataFrame(V_initial, index=s_labels, columns=component_labels)
print(df)



N_zeros = np.count_nonzero(V_initial == 0)

Nv = Ns*(Nc+1) - N_zeros ####################### why not Nv = np.count_nonzero(V_initial)
Nd = Nv- Ns - Nc

print ("\nThe number of total variable is",Nv)
print (colored("The number of design variable that you have designed",attrs=["bold"]),Nd)


# List all the unknown (waited to be resolved) variables  
unknown_variables = [element for element in all_variables if element not in given_variables]
number_unknowns = len(unknown_variables)

print("\nYou have given",len(given_variables)-N_zeros,"variables")

print ("\nThere are",number_unknowns,"unknown variable:")
print(*unknown_variables,sep="，")


# Check if you have given too many or didn't give enough design variables
if len(given_variables)-N_zeros > Nd:
    print(colored("You have given too many input variable","red"))
if len(given_variables)-N_zeros < Nd:
    print(colored("You didnt give enough input variables","red"))


             Flow rate  Oxygen Nitrogen Argon
stream-1_in         F1  0.2098   0.7808   y1A
stream-2_out       100     0.9     0.02   y2A
stream-3_out        F3     y3O      y3N     0

The number of total variable is 11
The number of design variable that you have designed 5

You have given 5 variables

There are 6 unknown variable:
F1，F3，y3O，y3N，y1A，y2A


In [33]:
given_variables

['F2', 'y1O', 'y2O', 'y1N', 'y2N', 'y3A']

In [51]:
df


,Flow rate,Oxygen,Nitrogen,Argon
stream-1_in,F1,0.2098,0.7808,y1A
stream-2_out,100,0.9,0.02,y2A
stream-3_out,F3,y3O,y3N,0


,Flow rate,Oxygen,Nitrogen,Argon
stream-1_in,F1,0.2098,0.7808,y1A
stream-2_out,100,0.9,0.02,y2A
stream-3_out,F3,y3O,y3N,0
